I think that VCFs are totally not needed here and instead I should mpileup at the loci of interest for each sample and then analyze that output.

In [1]:
source("../../BrusselSprouts/scripts/functions.R")
source("02_MpileupFunctions.r")
time_start=Sys.time()


Attaching package: 'dplyr'


The following objects are masked from 'package:data.table':

    between, first, last


The following objects are masked from 'package:stats':

    filter, lag


The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union




In [2]:
reference="/home/jnrunge/data/TRD/R64_nucl.fasta"

In [3]:
bam_files=list.files(path="~/data/trd/mapped_reads",pattern="JNRC[0-9]*.bam$",full.names = TRUE)
bam_files=naturalsort(bam_files)
bam_files

[1] "/home/jnrunge/data/trd/mapped_reads/YJNRC2.bam" 
[2] "/home/jnrunge/data/trd/mapped_reads/YJNRC17.bam"
[3] "/home/jnrunge/data/trd/mapped_reads/YJNRC18.bam"

In [9]:
Crosses=readxl::read_xlsx("~/data/trd/Crosses.xlsx", sheet=2)
head(Crosses<-Crosses[,c("Cross ID","Short name 1","Short name 2")])
tail(Crosses)

New names:
* `Is in Stock` -> `Is in Stock...4`
* `Is in Stock` -> `Is in Stock...6`


Cross ID,Short name 1,Short name 2
<chr>,<chr>,<chr>
YJNRC1,BAK,BET
YJNRC2,BAN,BTI
YJNRC3,AKH,BQC
YJNRC4,ABS,BEF
YJNRC5,BAK,BMK
YJNRC6,BAN,CRB


Cross ID,Short name 1,Short name 2
<chr>,<chr>,<chr>
YJNRC29,AAR,CPG
YJNRC30,ABG,CPG
YJNRC31,AKI,CPG
YJNRC32,ATE,CPG
YJNRC33,AAR,AKH
YJNRC34,AKH,AKQ


In [10]:
Crosses[Crosses$`Cross ID`=="YJNRC17",]

Cross ID,Short name 1,Short name 2
<chr>,<chr>,<chr>
YJNRC17,AVI,BET


In [5]:
cross=unlist(lapply(basename(bam_files), getFirst_v2, split="."))
cross=cross

In [6]:
cross

[1] "YJNRC2"  "YJNRC17" "YJNRC18"

In [7]:
getBase=function(x, which){
        base=as.numeric(substr(initial_GTs[x,5+which],1,1))
        if(base==0){
            return(initial_GTs[x,3])
        }
        V4=strsplit(initial_GTs[x,4],split = ",",fixed=TRUE)[[1]]
        return(V4[base])
    }

In [8]:
state=list()
while(sum(unlist(state)=="MP-done")!=length(cross))
    {
    

for(i in 1:length(cross)){
    state[[i]]="running"
    if(file.exists(paste0("~/data/trd/genotypes/",cross[i],".mpileup-readable.txt.gz"))){
        #next
        #state[[i]]="MP-done"
    }
    # get the opp. homo. loci from the main vcf
    cmd=paste0("sh ~/TRD/01_Mapping/02_FindOHLoci.sh ",Crosses$`Short name 1`[Crosses$`Cross ID`==cross[i]]," ",Crosses$`Short name 2`[Crosses$`Cross ID`==cross[i]]," ",cross[i])
    
    if(file.exists(paste0("~/data/TRD/Oppo-Homo-Pos/",cross[i],".running"))){
        if(file.mtime(paste0("~/data/TRD/Oppo-Homo-Pos/",cross[i],".running"))>=time_start){
            state[[i]]="OH-running"
        }
        }

    if(!file.exists(paste0("~/data/TRD/Oppo-Homo-Pos/",cross[i],".ohloci.gz"))){
         if(state[[i]]!="OH-running"){
            execute_cmd_sbatch(cmd,mem = "4gb", cpu="1", env="bwaetc", jobname = "GetOHLoci")
        }
        

        if(!file.exists(paste0("~/data/TRD/Oppo-Homo-Pos/",cross[i],".done"))){
            state[[i]]="OH-running"
            next
        }
        if(file.mtime(paste0("~/data/TRD/Oppo-Homo-Pos/",cross[i],".done"))<time_start){
            state[[i]]="OH-running"
                next
                }
    }
    
    state[[i]]="OH-done"

    initial_GTs=fread(paste0("~/data/TRD/Oppo-Homo-Pos/",cross[i],".ohloci.GT.gz"))
    initial_GTs$V5=unlist(lapply(initial_GTs$V5, getFirst_v2, split=":"))
    initial_GTs$V6=unlist(lapply(initial_GTs$V6, getFirst_v2, split=":"))
    initial_GTs$baseID1=unlist(lapply(1:nrow(initial_GTs), getBase, which=0))
    initial_GTs$baseID2=unlist(lapply(1:nrow(initial_GTs), getBase, which=1))
    initial_GTs=initial_GTs[,c("V1","V2","baseID1","baseID2")]
    head(initial_GTs)
    
    if(!file.exists(paste0("~/data/trd/genotypes/",cross[i],".mpileup.txt.gz"))){

        cmd=paste0("touch ",cross[i],".mpileup.running && gunzip ","~/data/TRD/Oppo-Homo-Pos/",cross[i],".ohloci.gz && cd ~/data/trd/genotypes && samtools mpileup --redo-BAQ --min-BQ 30 --min-MQ 30 -l ","~/data/TRD/Oppo-Homo-Pos/",cross[i],".ohloci",
                   " -o ",cross[i],".mpileup.txt -f ",reference," ",bam_files[i]," && gzip -f ",cross[i],".mpileup.txt && gzip -f ","~/data/TRD/Oppo-Homo-Pos/",cross[i],".ohloci && touch ",cross[i],".mpileup.done")
        
        if(file.exists(paste0("~/data/TRD/Oppo-Homo-Pos/",cross[i],".mpileup.running"))){
        if(file.mtime(paste0("~/data/TRD/Oppo-Homo-Pos/",cross[i],".mpileup.running"))>=time_start){
            state[[i]]="MP-running"
        }
        }
        if(state[[i]]!="MP-running"){
            execute_cmd_sbatch(cmd,mem = "4gb", cpu="1", env="bwaetc", jobname = "Mpileup")
        }

        

            if(!file.exists(paste0("~/data/trd/genotypes/",cross[i],".mpileup.done"))){
                state[[i]]="MP-running"
                next
            }
            if(file.mtime(paste0("~/data/trd/genotypes/",cross[i],".mpileup.done"))<time_start){
                state[[i]]="MP-running"
                    next
                    }
        }
    
    state[[i]]="MP-done"

    mpileup_df=fread(paste0("~/data/trd/genotypes/",cross[i],".mpileup.txt.gz"))
    mpileup_df=bind_cols(mpileup_df,stringr_count_bases(string_column = mpileup_df$V5, ref_column = mpileup_df$V3))
    mpileup_df=left_join(mpileup_df, initial_GTs, by=c("V1","V2"))
    mpileup_df$countID1=0
    mpileup_df$countID2=0
    colnames(mpileup_df)[7:10]=toupper(colnames(mpileup_df)[7:10])
    for(j in 1:nrow(mpileup_df)){
        mpileup_df$countID1[j]=mpileup_df[j,mpileup_df$baseID1[j]]
        mpileup_df$countID2[j]=mpileup_df[j,mpileup_df$baseID2[j]]
    }
    head(mpileup_df)
    colnames(mpileup_df)[1:2]=c("chr","pos")
    fwrite(mpileup_df[,c("chr","pos","countID1","countID2")],
          paste0("~/data/trd/genotypes/",cross[i],".mpileup-readable.txt.gz"))
}
    Sys.sleep(60)
    }
    
    

In [ ]:
state